# TensorFlow Tutorial #21
# 机器翻译

by [Magnus Erik Hvass Pedersen](http://www.hvass-labs.org/)/[GitHub中文](https://github.com/Hvass-Labs/TensorFlow-Tutorials-Chinese)
/ [GitHub](https://github.com/Hvass-Labs/TensorFlow-Tutorials) / [Videos on YouTube](https://www.youtube.com/playlist?list=PL9Hr9sNUjfsmEu1ZniY0XpHSzl5uihcXZ)

中文翻译[ZhouGeorge](https://github.com/ZhouGeorge)

# 介绍

教程 #20 展示了如何运用循环神经网络（RNN）去完成电影评论的情感分析。这份教程通过两个神经网络合并将这个想法扩展到人类语言的机器翻译。


你需要要熟悉TensorFlow，Keras和自然语言处理的基础，详见 #01 ， #03-C和 #20。

## 流程图

下面的流程图大致展示了神经网络是如何构造的。它被分为两个部分：一个编码器将源文本映射到“思维向量”（thought vector），它总结了文本的内容，然后将它输入到神经网络的第二个部分，将“思维向量”（thought vector）解码成目标文本。

神经网络不能直接作用到文本上，所以我们首先要用分词器将每一个词转换到整形代号（integer-token）。但是神经网络也不能直接作用在整数上，所以我们要用Embedding层将整形代号（integer-token）转换成浮点形的向量。embedding层与神经网络的其他部分一起训练，主要用于将相似情感的词映射到相似的浮点向量上。

举个例子，丹麦语 "der var engang"，是每一个通话开始的语句，字面的翻译是"there was once"，但是它通常被翻译成英语"once upon a time"。我们首先将全部的数据集转换成整形代号，所以 "der var engang" 变成了 [12, 54, 1097]。每一个整形代号（integer-tokens）被映射到一个嵌入向量（embedding-vector），例如有128个元素的，所以整形代号12可以变成例如[0.12, -0.56, ..., 1.19]，整形代号54可以变成例如 [0.39, 0.09, ..., -0.12].，这些嵌入（ embedding-vectors）向量之后可以被输入到有3个GRU层的循环神经网络。更多的细节见 #20.

最后一个GRU层输出单独的向量-即总结了原文本内容的“思维向量”-它然后作为解码部分中GUR单元的初始状态。

目标文本"once upon a time"被特殊的记号“ssss”和“eeee”填充，意味着文本的开始和结尾，所以整形代号的序列变成了[2, 337, 640, 9, 79, 3].。在训练过程中，解码器会以全部的序列作为输入，期望的输出序列是 [337, 640, 9, 79, 3] ，与上面的序列相同但时间偏移了一步。我们尝试教编码器去映射“思维向量”和起始代号"ssss"（整数2）到下一个词"once"（整形 337），然后将 "once"映射到"upon"（整数 640），诸如此类。


下面的流程图描述了主要的思想，但是没有展示出全部必须的细节，例如关于损失函数内容，它也有点复杂。


![Flowchart](images/21_machine_translation_flowchart.png)

## 导入

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import math
import os

我们需要从Keras中导入一些模块。

In [2]:
# from tf.keras.models import Model  # This does not work!
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Input, Dense, GRU, Embedding
from tensorflow.python.keras.optimizers import RMSprop
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

开发环境Python 3.6，等等：

In [3]:
tf.__version__

'1.5.0'

In [4]:
tf.keras.__version__

'2.1.2-tf'

## 加载数据

我将使用欧洲议会数据集，包含了大多数欧洲语言的句子对。这些数据是由欧盟创建的，将大量的通信翻译成欧盟成员国的语言。

http://www.statmt.org/europarl/

In [5]:
import europarl

在这份教程中我们使用了英语-丹麦语（English-Danish）的数据集，它包含了2百万个句子对。你可以通过改变language-code来使用另一种语言，具体见`europarl.py`中关于可用语言代码的列表。

In [6]:
language_code='da'

为了让解码器知道什么是一个序列的开始和结束，我们需要用不可能出现在数据集中的单词来标记每一个序列的开始和结束。

In [7]:
mark_start = 'ssss '
mark_end = ' eeee'

你可以根据喜好来改变数据文件的地址。

In [8]:
# data_dir = "data/europarl/"

如果你没有准备好数据，它会被自动下载和解压。

**注意 :English-Danish数据集大约有587 MB！**

In [9]:
europarl.maybe_download_and_extract(language_code=language_code)

Data has apparently already been downloaded and unpacked.


加载源语言的文本，这里使用丹麦语。

In [10]:
data_src = europarl.load_data(english=False,
                              language_code=language_code)

记载目标语言的文件，这里是英语。

In [11]:
data_dest = europarl.load_data(english=True,
                               language_code=language_code,
                               start=mark_start,
                               end=mark_end)

我们将建立一个将源语言（丹麦语）翻译到目标语言（英语）的模型。如果你想做一个逆向的翻译，你仅仅需要改变源数据和目标数据。

### 实例数据

数据是一列排序好的文本，源文文本和目标文本的索引是相匹配的。我们可以找个例子确认下翻译是否对应。


In [12]:
idx = 2

In [13]:
data_src[idx]

'Som De kan se, indfandt det store "år 2000-problem" sig ikke. Til gengæld har borgerne i en del af medlemslandene været ramt af meget forfærdelige naturkatastrofer.'

In [14]:
data_dest[idx]

"ssss Although, as you will have seen, the dreaded 'millennium bug' failed to materialise, still the people in a number of countries suffered a series of natural disasters that truly were dreadful. eeee"

###  数据中的错误

数据集中包括2百万个句子对。有些数据是不正确的。这个例子中出现了法语（或者是一些奇怪的语言），尽管里面也有丹麦语。

In [15]:
idx = 8002

In [16]:
data_src[idx]

'"Car il savait ce que cette foule en joie ignorait, et qu\'on peut lire dans les livres, que le bacille de la peste ne meurt ni ne disparaît jamais, qu\'il peut rester pendant des dizaines d\'années endormi dans les meubles et le linge, qu\'il attend patiemment dans les chambres, les caves, les malles, les mouchoirs et les paperasses, et que, peut-être, le jour viendrait où, pour le malheur et l\'enseignement des hommes, la peste réveillerait ses rats et les enverrait mourir dans une cité heureuse." (Thi han vidste det, som denne glade forsamling ikke vidste, og som man kan læse i bøger, at pestens bacille aldrig dør og aldrig forsvinder, at den kan sove i mange år i møbler og linned, at den venter tålmodigt i kamre, kældre, kufferter, lommetørklæder og papirer, og at den dag måske kommer, hvor pesten til menneskenes skade og oplysning vågner sine rotter og sender dem ud for at dø i en lykkelig by.)'

In [17]:
data_dest[idx]

'ssss "He knew what those jubilant crowds did not know but could have learned from books: that the plague bacillus never dies or disappears for good; that it can lie dormant for years and years in furniture and linen-chests; that it bides its time in bedrooms, cellars, trunks, and bookshelves; and that perhaps the day would come when, for the bane and the enlightening of men, it would rouse up its rats again and send them forth to die in a happy city." eeee'

## 分词器

神经网络不能直接作用到文本。我们需要用两步处理，将文本转换成可以出入神经网络的数字。第一步是将文本词汇转换成整形代号（integer-tokens）。第二步是利用embedding层将整形代号转换成浮点值的向量。更多的细节见教程 #20.

设置词汇表的最大词汇数。这意味着我们将用数据集中（例如）10000个频率最高的词汇。这里我们对源语言和目标语言中设置了相同的大小，但这其实也是可以设置成不同的大小。


In [18]:
num_words = 10000

我们需要的功能比Keras中分词器类能提供的功能更多，所以我们将它们封装起来。

In [19]:
class TokenizerWrap(Tokenizer):
    """Wrap the Tokenizer-class from Keras with more functionality."""
    
    def __init__(self, texts, padding,
                 reverse=False, num_words=None):
        """
        :param texts: List of strings. This is the data-set.
        :param padding: Either 'post' or 'pre' padding.
        :param reverse: Boolean whether to reverse token-lists.
        :param num_words: Max number of words to use.
        """

        Tokenizer.__init__(self, num_words=num_words)

        # Create the vocabulary from the texts.
        self.fit_on_texts(texts)

        # Create inverse lookup from integer-tokens to words.
        self.index_to_word = dict(zip(self.word_index.values(),
                                      self.word_index.keys()))

        # Convert all texts to lists of integer-tokens.
        # Note that the sequences may have different lengths.
        self.tokens = self.texts_to_sequences(texts)

        if reverse:
            # Reverse the token-sequences.
            self.tokens = [list(reversed(x)) for x in self.tokens]
        
            # Sequences that are too long should now be truncated
            # at the beginning, which corresponds to the end of
            # the original sequences.
            truncating = 'pre'
        else:
            # Sequences that are too long should be truncated
            # at the end.
            truncating = 'post'

        # The number of integer-tokens in each sequence.
        self.num_tokens = [len(x) for x in self.tokens]

        # Max number of tokens to use in all sequences.
        # We will pad / truncate all sequences to this length.
        # This is a compromise so we save a lot of memory and
        # only have to truncate maybe 5% of all the sequences.
        self.max_tokens = np.mean(self.num_tokens) \
                          + 2 * np.std(self.num_tokens)
        self.max_tokens = int(self.max_tokens)

        # Pad / truncate all token-sequences to the given length.
        # This creates a 2-dim numpy matrix that is easier to use.
        self.tokens_padded = pad_sequences(self.tokens,
                                           maxlen=self.max_tokens,
                                           padding=padding,
                                           truncating=truncating)

    def token_to_word(self, token):
        """Lookup a single word from an integer-token."""

        word = " " if token == 0 else self.index_to_word[token]
        return word 

    def tokens_to_string(self, tokens):
        """Convert a list of integer-tokens to a string."""

        # Create a list of the individual words.
        words = [self.index_to_word[token]
                 for token in tokens
                 if token != 0]
        
        # Concatenate the words to a single string
        # with space between all the words.
        text = " ".join(words)

        return text
    
    def text_to_tokens(self, text, reverse=False, padding=False):
        """
        Convert a single text-string to tokens with optional
        reversal and padding.
        """

        # Convert to tokens. Note that we assume there is only
        # a single text-string so we wrap it in a list.
        tokens = self.texts_to_sequences([text])
        tokens = np.array(tokens)

        if reverse:
            # Reverse the tokens.
            tokens = np.flip(tokens, axis=1)

            # Sequences that are too long should now be truncated
            # at the beginning, which corresponds to the end of
            # the original sequences.
            truncating = 'pre'
        else:
            # Sequences that are too long should be truncated
            # at the end.
            truncating = 'post'

        if padding:
            # Pad and truncate sequences to the given length.
            tokens = pad_sequences(tokens,
                                   maxlen=self.max_tokens,
                                   padding='pre',
                                   truncating=truncating)

        return tokens

现在我们可以创建源语言的分词器。注意，我们选择('pre')来对序列填充0。我们也反转了序列的代号，研究表明这样可以提高性能，因为编码器所看到的最后一个词与解码器产生的第一个词相匹配，因此短期依赖性被认为能更精确地建模。

In [20]:
%%time
tokenizer_src = TokenizerWrap(texts=data_src,
                              padding='pre',
                              reverse=True,
                              num_words=num_words)

CPU times: user 2min 17s, sys: 608 ms, total: 2min 17s
Wall time: 2min 17s


现在可以创建目标语言的分词器。我们对源语言和目标语言都需要创建分词器是因为它们的词汇表是不同的。注意分词器不会反转序列，它会在数组的末尾('post')填0。

In [21]:
%%time
tokenizer_dest = TokenizerWrap(texts=data_dest,
                               padding='post',
                               reverse=False,
                               num_words=num_words)

CPU times: user 1min 42s, sys: 492 ms, total: 1min 42s
Wall time: 1min 42s


为填充过的代号序列定义变量。这些只是是2维的整形代号（integer-tokens）的numpy数组

注意，对于源语言和目标语言，序列的长度是不同。这是因为具有相同含义的文本在这两种语言中可能有不同数量的单词。

而且，我们在对原始文本分词时为了节约内存做了一个妥协。这意味着我们只截取了大约5%的文本。

In [22]:
tokens_src = tokenizer_src.tokens_padded
tokens_dest = tokenizer_dest.tokens_padded
print(tokens_src.shape)
print(tokens_dest.shape)

(1968800, 47)
(1968800, 55)


在目标语言中，用于标记文本开始的标记整形代号。

In [23]:
token_start = tokenizer_dest.word_index[mark_start.strip()]
token_start

2

在目标语言中，用于标记文本结束的标记整形代号。

In [24]:
token_end = tokenizer_dest.word_index[mark_end.strip()]
token_end

3

### 代号序列的例子

下面是分词器的输出，注意，在序列的开始先用0填充了。

In [25]:
idx = 2

In [26]:
tokens_src[idx]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 3069,
       3374,   43,    7, 1386,  108, 1995,    7,  178,    9,    3,  302,
         19, 2076,    8,   20,   39,  285,  499,   69,  136,    5,  166,
         24,   10,   13], dtype=int32)

我们可以通过整形代号转换到相应的单词来重构原始的文本。

In [27]:
tokenizer_src.tokens_to_string(tokens_src[idx])

'naturkatastrofer forfærdelige meget af ramt været medlemslandene af del en i borgerne har gengæld til ikke sig problem 2000 år store det se kan de som'

这个文本其实已经被反转了，数据集中原始的文本如下：

In [28]:
data_src[idx]

'Som De kan se, indfandt det store "år 2000-problem" sig ikke. Til gengæld har borgerne i en del af medlemslandene været ramt af meget forfærdelige naturkatastrofer.'

下面是目标语言的对应文本的的整形代号序列。注意它的末尾被0填充。


In [29]:
tokens_dest[idx]

array([   2,  404,   19,   43,   26,   20,  618,    1, 1451,    5, 9785,
        174,    1,   81,    7,    9,  214,    4,   67, 2200,    9, 1596,
          4,  892, 1762,    8, 1480,  107, 5494,    3,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
      dtype=int32)

我们可以通过整形代号转换到相应的单词来重构原始的文本。

In [30]:
tokenizer_dest.tokens_to_string(tokens_dest[idx])

'ssss although as you will have seen the failed to materialise still the people in a number of countries suffered a series of natural disasters that truly were dreadful eeee'

将它与数据集中原始的文本作对比，除了标点符号和一些单词，如 "dreaded millennium bug"之外，几乎是一样的。这是因为我们的词汇表只包含了数据集中10000个频率最多的词，而这3个词出现的频率很低，没有被包括到词汇表中，所以它们被忽略了。

In [31]:
data_dest[idx]

"ssss Although, as you will have seen, the dreaded 'millennium bug' failed to materialise, still the people in a number of countries suffered a series of natural disasters that truly were dreadful. eeee"

### 训练数据

现在数据集已经被转换成整形代号（integer-tokens）的序列，它经过填充和截断处理并保存为numpy数组，我们可以用它来训练神经网络。

编码器输入的仅仅是由分词器生成的被填充和截断的整形代号序列的numpy数组：

In [32]:
encoder_input_data = tokens_src

解码器的输入和输出数据是相同的，除了时间偏移了一步。我们可以用相同的数组来节省内存，仅仅需要对内存中相同的数据的采用不同的获取方式。

In [33]:
decoder_input_data = tokens_dest[:, :-1]
decoder_input_data.shape

(1968800, 54)

In [34]:
decoder_output_data = tokens_dest[:, 1:]
decoder_output_data.shape

(1968800, 54)

举个例子，下面的代号序列是相同的，除了它们被移动了一个时间步长。

In [35]:
idx = 2

In [36]:
decoder_input_data[idx]

array([   2,  404,   19,   43,   26,   20,  618,    1, 1451,    5, 9785,
        174,    1,   81,    7,    9,  214,    4,   67, 2200,    9, 1596,
          4,  892, 1762,    8, 1480,  107, 5494,    3,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
      dtype=int32)

In [37]:
decoder_output_data[idx]

array([ 404,   19,   43,   26,   20,  618,    1, 1451,    5, 9785,  174,
          1,   81,    7,    9,  214,    4,   67, 2200,    9, 1596,    4,
        892, 1762,    8, 1480,  107, 5494,    3,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
      dtype=int32)

如果我们用分词器将这些序列转换回文本，我们可以看到它们是相同的，除了一个作为文本开始的标记'ssss'。

In [38]:
tokenizer_dest.tokens_to_string(decoder_input_data[idx])

'ssss although as you will have seen the failed to materialise still the people in a number of countries suffered a series of natural disasters that truly were dreadful eeee'

In [1]:
tokenizer_dest.tokens_to_string(decoder_output_data[idx])

NameError: name 'tokenizer_dest' is not defined

## 创建神经网络

### 创建编码器

首先我们创建神经网络的编码部分，它将整形代号序列映射到“思维向量”（thought vector）。我们将用Keras的函数式（functional）来API完成，我们首先创建神经网络所有层的对象，然后将它们连接，这比Keras中序列式（sequential）API更灵活，当我们尝试更复杂的架构或将连接编码器和解码器的方法时，这个方法很有用。

这是编码器的输入，它接受按批次的整形代号序列。`None`表示序列的长度可以是任意的。

In [40]:
encoder_input = Input(shape=(None, ), name='encoder_input')

这是embedding层的输出向量的长度，embedding层将整形代号（integer-tokens ）映射成-1到1的向量的值。所以有相同含义的词映射的向量是相似的。更详细的解释见教程 #20。

In [41]:
embedding_size = 128

这是embedding层。

In [42]:
encoder_embedding = Embedding(input_dim=num_words,
                              output_dim=embedding_size,
                              name='encoder_embedding')

这是GRU内部状态的大小。编码和解码都用了这个大小。

In [43]:
state_size = 512

创建3个GRU层，用于将嵌入向量（embedding-vectors）映射到一个单独的“思维向量” （thought vector），这个向量总结了输入文本的内容。注意最后一个GRU层没有返回序列。

In [44]:
encoder_gru1 = GRU(state_size, name='encoder_gru1',
                   return_sequences=True)
encoder_gru2 = GRU(state_size, name='encoder_gru2',
                   return_sequences=True)
encoder_gru3 = GRU(state_size, name='encoder_gru3',
                   return_sequences=False)

用于连接编码所有层的辅助函数。

In [45]:
def connect_encoder():
    # Start the neural network with its input-layer.
    net = encoder_input
    
    # Connect the embedding-layer.
    net = encoder_embedding(net)

    # Connect all the GRU-layers.
    net = encoder_gru1(net)
    net = encoder_gru2(net)
    net = encoder_gru3(net)

    # This is the output of the encoder.
    encoder_output = net
    
    return encoder_output

注意编码器是如何使用最后一个GRU层的正常输出作为“思维向量”（thought vector）。研究论文经常使用编码器的最后一个循环层的内部状态作为“思维向量”（thought vector）。但这让实现变得更复杂了并且当使用GRU时也是不需要的。但是如果你使用的是LSTM，那么使用LSTM的内部状态作为“思维向量”是必须的，因为它有两个内部向量，我们需要初始化编码器的LSTM单元的两个内部状态。

我们现在可以用这个函数完成编码器层的连接，它在将来会被连接到解码器上。

In [46]:
encoder_output = connect_encoder()

Instructions for updating:
keep_dims is deprecated, use keepdims instead


### 创建解码器

创建解码部分，它将“思维向量”映射到整形代号序列。

解码器有两个输入。首先它需要有编码器产生的总结了输入文本内容的“思维向量”（thought vector）。


In [47]:
decoder_initial_state = Input(shape=(state_size,),
                              name='decoder_initial_state')

解码器也需要一个整形代号序列作为输入。在训练时，我们将提供完整的整形代号序列，例如对于文本"ssss once upon a time eeee"。

当我们在翻译新的输入文本时，我们首先喂入序列中"ssss"的整形代号（它是一个文本开始的标记），然后与编码器的“思维向量”结合在一起，解码器将有望产生正确的下一个单词，例如"once"。

In [48]:
decoder_input = Input(shape=(None, ), name='decoder_input')

这是embedding层，它将整形代号转换成-1到1的实值向量。注意我们对编码器和解码器有不同的embedding层，因为我们有两个不同词汇表和两个不同分词器对应源语言和目标语言。


In [49]:
decoder_embedding = Embedding(input_dim=num_words,
                              output_dim=embedding_size,
                              name='decoder_embedding')

创建解码器的3个GRU。注意它们都返回序列，因为我们最后想要输出一个整形代号的序列，它可以被转换成文本序列。

In [50]:
decoder_gru1 = GRU(state_size, name='decoder_gru1',
                   return_sequences=True)
decoder_gru2 = GRU(state_size, name='decoder_gru2',
                   return_sequences=True)
decoder_gru3 = GRU(state_size, name='decoder_gru3',
                   return_sequences=True)

GRU层最后输出张量的形状是`[batch_size, sequence_length, state_size]`，每一个"词"被编码成长度为`state_size`的向量。我们需要将这些转换到可以用我们的词汇表解释的整形代号序列，

一种方法是将GRU的输出转换成独热编码的数组。它能起效，但是相当浪费，因为词汇表有10000个词，我们需要有10000个元素的向量，所以我们可以选择最高元素的索引作为整形代号（integer-token）。

注意激活函数被设置为`linear`来代替`softmax`，就像我们通常使用于独热编码的输出一样，因为Keras中有似乎有一个bug，所以我们需要自己定制一个损失函数，下面会详细介绍。

In [51]:
decoder_dense = Dense(num_words,
                      activation='linear',
                      name='decoder_output')

解码器采用Keras的函数式（functional）API搭建，它允许层的连接更灵活，例如嫁接不同输入到解码器中。这是非常有用的，因为我们必须直接将解码器和编码器相连，但是我们也会连接解码器和另一输入，这样我们可以独立运行它。


这个函数将解码器的所有层连接到GRU层的初始状态值的输入。


In [52]:
def connect_decoder(initial_state):
    # Start the decoder-network with its input-layer.
    net = decoder_input

    # Connect the embedding-layer.
    net = decoder_embedding(net)
    
    # Connect all the GRU-layers.
    net = decoder_gru1(net, initial_state=initial_state)
    net = decoder_gru2(net, initial_state=initial_state)
    net = decoder_gru3(net, initial_state=initial_state)

    # Connect the final dense layer that converts to
    # one-hot encoded arrays.
    decoder_output = decoder_dense(net)
    
    return decoder_output

### 连接和创建模型

我们可以用不同的方式连接编码器和解码器。

首先我们将编码器和解码器直接连接，所以它们成了一整个模型，可以用端到端的方式训练。这意味着解码器GRU单元的初始状态被设置成编码器是输出。



In [53]:
decoder_output = connect_decoder(initial_state=encoder_output)

model_train = Model(inputs=[encoder_input, decoder_input],
                    outputs=[decoder_output])

然后我们创建给编码器单独建立一个模型。主要用于将整形代号序列映射到总结了文本内容的“思维向量”。


In [54]:
model_encoder = Model(inputs=[encoder_input],
                      outputs=[encoder_output])

然后我们创建一个单独解码器。它允许我们直接输入解码器的GRU单元初始状态。


In [55]:
decoder_output = connect_decoder(initial_state=decoder_initial_state)

model_decoder = Model(inputs=[decoder_input, decoder_initial_state],
                      outputs=[decoder_output])

注意到这些模型的编码器和解码器用了相同的权重和变量。我们仅仅改变它们的连接方式。所以一旦完整的模型被训练好了，我们可以用训练好的权重分别运行编码器和解码器。


### 损失函数

解码器的输出是独热编码的数组序列。为了训练解码器，我们需要提供我们希望看到解码器输出的独热编码数组，然后使用像交叉熵这样的损失函数去训练解码器产生期望的输出。

然而，我们的数据集包含整形代号，而不是独热编码数组。每一个独热编码数组有10000个元素，所以将全部的数据集转换成独热数组是相当浪费的。

一个好解决方法是使用稀疏的交叉熵（sparse cross-entropy）损失函数，它的内部完成了从整数到独热数组的转换。不幸的是，当和循环神经网络一起使用的时候，Keras似乎存在一个bug，所以下面的方式不起效：


In [56]:
# model_train.compile(optimizer=optimizer,
#                     loss='sparse_categorical_crossentropy')

解码器输出3阶形状 `[batch_size, sequence_length, num_words]`是张量，它包含了一批长度为`num_words`的独热数组的序列。我们将它与包含了整形序列的2阶`[batch_size, sequence_length]`张量作比较。

比较由Tensorflow中的稀疏交叉熵（sparse-cross-entropy）直接完成。下面还有一些需要注意的事项。

首先，为了提高数值的稳定性，损失函数在内部计算softamax，这也是为什么我们在解码器网络最后的全连接层中用了线性激活函数。

第二，TensoFlow的损失函数输出的是形状为`[batch_size, sequence_length]`的二阶张量。但是它最终要被转换为一个单独标量值，它的梯度可以被TensorFlow推导并可以用梯度下降来优化。Keras支持在批处理中对损失值进行加权，但描述的不清楚，所以为了确保我们计算的损失函数经过了整个批处理和整个序列，我们手工计算损失平均值。


In [57]:
def sparse_cross_entropy(y_true, y_pred):
    """
    Calculate the cross-entropy loss between y_true and y_pred.
    
    y_true is a 2-rank tensor with the desired output.
    The shape is [batch_size, sequence_length] and it
    contains sequences of integer-tokens.

    y_pred is the decoder's output which is a 3-rank tensor
    with shape [batch_size, sequence_length, num_words]
    so that for each sequence in the batch there is a one-hot
    encoded array of length num_words.
    """

    # Calculate the loss. This outputs a
    # 2-rank tensor of shape [batch_size, sequence_length]
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y_true,
                                                          logits=y_pred)

    # Keras may reduce this across the first axis (the batch)
    # but the semantics are unclear, so to be sure we use
    # the loss across the entire 2-rank tensor, we reduce it
    # to a single scalar with the mean function.
    loss_mean = tf.reduce_mean(loss)

    return loss_mean

### 编译训练模型

我们在之前的教程中用了Adam优化器，但是在一些试验中发现它与循环神经网络有冲突。RMSprop在这似乎表现的更好一些。


In [58]:
optimizer = RMSprop(lr=1e-3)

Keras在这里似乎又有一个bug，它不能自动的推断正确的编码器输出的形状。因此我们需要手动创建一个占位符变量来作为解码器的输出。它的形状被设置成`(None, None)`，意味着这批可以有任意数量的序列，任意数量的整形代号。


In [59]:
decoder_target = tf.placeholder(dtype='int32', shape=(None, None))

我们现在可以用自己定制的损失函数编译模型。

In [60]:
model_train.compile(optimizer=optimizer,
                    loss=sparse_cross_entropy,
                    target_tensors=[decoder_target])

Instructions for updating:
keep_dims is deprecated, use keepdims instead


### 回调函数
在训练时，我们希望保存checkpoints并将进展记录到TensorBoard中，所以我们用了Keras中的相应回调。

下面是在训练时保存checkpoints的回调。


In [61]:
path_checkpoint = '21_checkpoint.keras'
callback_checkpoint = ModelCheckpoint(filepath=path_checkpoint,
                                      monitor='val_loss',
                                      verbose=1,
                                      save_weights_only=True,
                                      save_best_only=True)

下面是用于在验证集上模型表现的更糟时停止优化的回调。

In [62]:
callback_early_stopping = EarlyStopping(monitor='val_loss',
                                        patience=3, verbose=1)

下面是在训练时写TensorBoard日志的回调。

In [63]:
callback_tensorboard = TensorBoard(log_dir='./21_logs/',
                                   histogram_freq=0,
                                   write_graph=False)

In [64]:
callbacks = [callback_early_stopping,
             callback_checkpoint,
             callback_tensorboard]

### 加载 Checkpoint

我们可以重新加载最后保存的checkpoint ，所以当我们每当我们想要使用这个模型时，不需要再训练它。

In [65]:
try:
    model_train.load_weights(path_checkpoint)
except Exception as error:
    print("Error trying to load checkpoint.")
    print(error)

### 训练模型

我们将数据封装在命名的字典中，所以我们确认数据被正确地分配给模型的输入和输出

In [66]:
x_data = \
{
    'encoder_input': encoder_input_data,
    'decoder_input': decoder_input_data
}

In [67]:
y_data = \
{
    'decoder_output': decoder_output_data
}

我们想要10000序列的验证集，但是Keras需要这个数字是一个分数。

In [68]:
validation_split = 10000 / len(encoder_input_data)
validation_split

0.0050792360828931325

现在我们可以训练这个模型。在GTX 1070 GPU上训练一个epoch大约花费1小时。在训练时你可能需要运行10个epoch或者更多。在10个epochs后，在训练集上损失大约是1.10，在验证集上的损失大约是1.15。

注意这个被选为640（512+128）的批大小，当GPU内存现在在8GB时，它能让GPU接近100%的运行。


In [ ]:
model_train.fit(x=x_data,
                y=y_data,
                batch_size=640,
                epochs=10,
                validation_split=validation_split,
                callbacks=callbacks)

## 翻译文本

这个函数将一个文本从源语言转换到目标语言并且可选地打印正确的翻译。


In [69]:
def translate(input_text, true_output_text=None):
    """Translate a single text-string."""

    # Convert the input-text to integer-tokens.
    # Note the sequence of tokens has to be reversed.
    # Padding is probably not necessary.
    input_tokens = tokenizer_src.text_to_tokens(text=input_text,
                                                reverse=True,
                                                padding=True)
    
    # Get the output of the encoder's GRU which will be
    # used as the initial state in the decoder's GRU.
    # This could also have been the encoder's final state
    # but that is really only necessary if the encoder
    # and decoder use the LSTM instead of GRU because
    # the LSTM has two internal states.
    initial_state = model_encoder.predict(input_tokens)

    # Max number of tokens / words in the output sequence.
    max_tokens = tokenizer_dest.max_tokens

    # Pre-allocate the 2-dim array used as input to the decoder.
    # This holds just a single sequence of integer-tokens,
    # but the decoder-model expects a batch of sequences.
    shape = (1, max_tokens)
    decoder_input_data = np.zeros(shape=shape, dtype=np.int)

    # The first input-token is the special start-token for 'ssss '.
    token_int = token_start

    # Initialize an empty output-text.
    output_text = ''

    # Initialize the number of tokens we have processed.
    count_tokens = 0

    # While we haven't sampled the special end-token for ' eeee'
    # and we haven't processed the max number of tokens.
    while token_int != token_end and count_tokens < max_tokens:
        # Update the input-sequence to the decoder
        # with the last token that was sampled.
        # In the first iteration this will set the
        # first element to the start-token.
        decoder_input_data[0, count_tokens] = token_int

        # Wrap the input-data in a dict for clarity and safety,
        # so we are sure we input the data in the right order.
        x_data = \
        {
            'decoder_initial_state': initial_state,
            'decoder_input': decoder_input_data
        }

        # Note that we input the entire sequence of tokens
        # to the decoder. This wastes a lot of computation
        # because we are only interested in the last input
        # and output. We could modify the code to return
        # the GRU-states when calling predict() and then
        # feeding these GRU-states as well the next time
        # we call predict(), but it would make the code
        # much more complicated.

        # Input this data to the decoder and get the predicted output.
        decoder_output = model_decoder.predict(x_data)

        # Get the last predicted token as a one-hot encoded array.
        token_onehot = decoder_output[0, count_tokens, :]
        
        # Convert to an integer-token.
        token_int = np.argmax(token_onehot)

        # Lookup the word corresponding to this integer-token.
        sampled_word = tokenizer_dest.token_to_word(token_int)

        # Append the word to the output-text.
        output_text += " " + sampled_word

        # Increment the token-counter.
        count_tokens += 1

    # Sequence of tokens output by the decoder.
    output_tokens = decoder_input_data[0]
    
    # Print the input-text.
    print("Input text:")
    print(input_text)
    print()

    # Print the translated output-text.
    print("Translated text:")
    print(output_text)
    print()

    # Optionally print the true translated text.
    if true_output_text is not None:
        print("True output text:")
        print(true_output_text)
        print()

### 举例

从训练数据中翻译一个文本。翻译的相当不错。它与训练数据中翻译不一样，但是实际内容是相似的。


In [70]:
idx = 3
translate(input_text=data_src[idx],
          true_output_text=data_dest[idx])

Input text:
De har udtrykt ønske om en debat om dette emne i løbet af mødeperioden.

Translated text:
 you have expressed a wish for a debate on this matter during the part session eeee

True output text:
ssss You have requested a debate on this subject in the course of the next few days, during this part-session. eeee



下面是另一个例子，它也是一个合理的翻译，尽管它错误的翻译了自然灾害。注意"countries of the European Union" 被翻译为 "member states"，在这个文本中他们是同义词。

In [71]:
idx = 4
translate(input_text=data_src[idx],
          true_output_text=data_dest[idx])

Input text:
I mellemtiden ønsker jeg - som også en del kolleger har anmodet om - at vi iagttager et minuts stilhed til minde om ofrene for bl.a. stormene i de medlemslande, der blev ramt.

Translated text:
 in the meantime i also asked for a minute's silence on the memory of victims of the atrocities that have been committed in the member states eeee

True output text:
ssss In the meantime, I should like to observe a minute' s silence, as a number of Members have requested, on behalf of all the victims concerned, particularly those of the terrible storms, in the various countries of the European Union. eeee



我们将训练集中两个文本连接起来。模型先将合并的文本输入到编码器并似乎产生了对两个文本总结相当好的“思维向量”，所以解码器可以产生合理的翻译。

In [72]:
idx = 3
translate(input_text=data_src[idx] + data_src[idx+1],
          true_output_text=data_dest[idx] + data_dest[idx+1])

Input text:
De har udtrykt ønske om en debat om dette emne i løbet af mødeperioden.I mellemtiden ønsker jeg - som også en del kolleger har anmodet om - at vi iagttager et minuts stilhed til minde om ofrene for bl.a. stormene i de medlemslande, der blev ramt.

Translated text:
 you have expressed a wish for a vote on this question during the vote on thursday and in the end i would also like to ask you to pay tribute to the memory of a tragedy in the case of the victims of the various member states eeee

True output text:
ssss You have requested a debate on this subject in the course of the next few days, during this part-session. eeeessss In the meantime, I should like to observe a minute' s silence, as a number of Members have requested, on behalf of all the victims concerned, particularly those of the terrible storms, in the various countries of the European Union. eeee



如果我们反转这两个文本的顺序，对于后一种文本来说，含义并不十分清楚。


In [73]:
idx = 3
translate(input_text=data_src[idx+1] + data_src[idx],
          true_output_text=data_dest[idx+1] + data_dest[idx])

Input text:
I mellemtiden ønsker jeg - som også en del kolleger har anmodet om - at vi iagttager et minuts stilhed til minde om ofrene for bl.a. stormene i de medlemslande, der blev ramt.De har udtrykt ønske om en debat om dette emne i løbet af mødeperioden.

Translated text:
 in the meantime i would also like to ask you to remember that we have received a silence on the victims of the floods in the member states of the european union which have been particularly sensitive to this debate in the house eeee

True output text:
ssss In the meantime, I should like to observe a minute' s silence, as a number of Members have requested, on behalf of all the victims concerned, particularly those of the terrible storms, in the various countries of the European Union. eeeessss You have requested a debate on this subject in the course of the next few days, during this part-session. eeee



这是我自己编的例子。翻译的结果相当糟糕


In [74]:
translate(input_text="der var engang et land der hed Danmark",
          true_output_text='Once there was a country named Denmark')

Input text:
der var engang et land der hed Danmark

Translated text:
 there was a country that denmark was once again eeee

True output text:
Once there was a country named Denmark



我再编一个。尽管它是更复杂的文本，翻译的还好了一些。

In [75]:
translate(input_text="Idag kan man læse i avisen at Danmark er blevet fornuftigt",
          true_output_text="Today you can read in the newspaper that Denmark has become sensible.")

Input text:
Idag kan man læse i avisen at Danmark er blevet fornuftigt

Translated text:
 can you read in the newspapers that denmark has been sensible eeee

True output text:
Today you can read in the newspaper that Denmark has become sensible.



这是丹麦歌曲中的一个文本。它在丹麦语中没有很多含义。翻译的支离破碎，因为其中的几个词不在词汇表中。


In [76]:
translate(input_text="Hvem spæner ud af en butik og tygger de stærkeste bolcher?",
          true_output_text="Who runs out of a shop and chews the strongest bon-bons?")

Input text:
Hvem spæner ud af en butik og tygger de stærkeste bolcher?

Translated text:
 who is by a and by the powerful eeee

True output text:
Who runs out of a shop and chews the strongest bon-bons?



## 总结

这份教程展示了用两循环神经网络搭建编码/解码模型去完成人类语言的机器翻译的基本思想。用了欧盟Europarl数据集做了演示。

这个模型可以对一些文本产生合理的翻译，但对另一些文本则不适用。更好的网络结构和训练更多epoch可能可以提高性能。现在已知有非常多先进的模型可以提高翻译的质量。

然而，注意到这些模型并没有真正理解人类的语言是很重要的。这些模型不知道这些词的实际含义。这些模型仅仅是非常高级的函数近似器，它们可以在整数代号的序列之间进行映射。


## 练习

下面使一些可能会让你提升TensorFlow技能的一些建议练习。为了学习如何更合适地使用TensorFlow，实践经验是很重要的。

在你对这个Notebook进行修改之前，可能需要先备份一下。

* 接着训练10个epoch。翻译是否有所提高？
* 增加词汇表的大小。翻译是否有所提高？源语言和目标语言词汇表的大小不同有什么意义吗？
* 找到另一个数据集并与Europarl一起使用。
* 改变网络结构，例如改变GRU层的状态大小，GRU的层数，embedding层的大小等等。翻译是否有所提高？
* 用教程 #19中的超参数优化去自动找到最好的超参数。
* 翻译文本时，你能不能把解码器的输出样本看作是概率分布，来代替使用 `np.argmax()`对下一个整形代号的采样？注意解码器的输出不受softmax的限制，所以你必须先把它变成概率分布。
* 你可以通过做这个采样来生成多个序列吗？你能找到一种方法来选择这些不同序列中最好的吗？
* 禁用源语言中单词的反转。翻译是否有所提高？
* 什么是双向Bi-DirectionalGRU，你可以在这里使用它吗？
* 我们用编码器的GRU的输出作为解码器GRU的初始状态。研究文献通常使用的是LSTM而不是GRU，所以他们使用了编码器的状态来代替它的输出作为解码器的初始状态。你可以将代码改写成用编码器的状态作为解码器的初始状态吗？有理由这么做吗，或者编码器的输出足以作为解码器的初始状态？
* 是否有可能将多个编码器和解码器连接到一个神经网络中，这样你就可以用不同的语言进行训练并允许直接翻译，例如丹麦语到波兰语，德语和法语。
* 向朋友解释程序是如何工作的

## License (MIT)

Copyright (c) 2018 by [Magnus Erik Hvass Pedersen](http://www.hvass-labs.org/)

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.